In [84]:
import selenium
from selenium import webdriver

In [85]:
Driver_path='C:\\Users\\Avi\\Desktop\\images\\chromedriver'

In [86]:

wd = webdriver.Chrome(executable_path=Driver_path)

In [90]:

def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.rg_ic")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.irc_mi')
            for actual_image in actual_images:
                if actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(1)
            load_more_button = wd.find_element_by_css_selector(".ksb")
            if load_more_button:
                wd.execute_script("document.querySelector('.ksb').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [91]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [92]:
import PIL

In [93]:

def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [94]:

def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=100):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [95]:
import os
from time import sleep
import time
import requests
import io
import PIL.Image
from PIL import Image
import hashlib

In [96]:
search_term='vegetable peels'
search_and_download(search_term=search_term,
                   driver_path=Driver_path)

Found: 200 search results. Extracting links from 0:200
Found: 100 image links, done!
SUCCESS - saved https://thumbs.dreamstime.com/z/vegetable-peels-38065111.jpg - as ./images\vegetable_peels\cce4f7145f.jpg
SUCCESS - saved https://static.toiimg.com/photo/65231303.cms - as ./images\vegetable_peels\bf7a759932.jpg
SUCCESS - saved https://edibleaspen.ediblecommunities.com/sites/default/files/styles/recipe_banner/public/images/recipe/vegetable-peel-chips-1.jpg?itok=-KP4_hPK - as ./images\vegetable_peels\a0247c24ea.jpg
SUCCESS - saved https://trashmagination.com/wp-content/uploads/2019/04/julia-kouzenkov-1360704-unsplash-e1555030727197-700x400.jpg - as ./images\vegetable_peels\dbb72bcbd1.jpg
SUCCESS - saved https://i.ndtvimg.com/i/2016-12/fruits-and-vegetables_620x350_81482747568.jpg - as ./images\vegetable_peels\9a59bd0b83.jpg
SUCCESS - saved https://media.istockphoto.com/photos/fruit-and-vegetable-waste-recycle-on-white-picture-id905480508?k=6&m=905480508&s=612x612&w=0&h=BdTB6OQAAat56_K00n

SUCCESS - saved https://www.berkeleywellness.com/sites/default/files/field/image/400-07035993c-Masterfile-dodoman_774_500.jpg - as ./images\vegetable_peels\5811b46e71.jpg
SUCCESS - saved https://www.1mhealthtips.com/wp-content/uploads/2015/10/i-PqdqhrC-M3.jpg - as ./images\vegetable_peels\e39526a649.jpg
SUCCESS - saved https://cdn7.dissolve.com/p/D9_17_019/D9_17_019_1200.jpg - as ./images\vegetable_peels\8d3ec78dd6.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/waste-based-cooking-elevated-view-260nw-621960833.jpg - as ./images\vegetable_peels\42ccd62953.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/compost-pile-hill-kitchen-scraps-260nw-170960243.jpg - as ./images\vegetable_peels\df499ec861.jpg
SUCCESS - saved https://femina.wwmindia.com/content/2018/jan/lead_shutterstock_75079306_in_1517311744.jpg - as ./images\vegetable_peels\75deef8995.jpg
SUCCESS - saved https://www.rd.com/wp-content/uploads/2019/06/peel-potato-760x506.jpg - as ./images\vegetable_p